## 3a

In [1]:
#Import libraries
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import *
from functools import reduce

In [2]:
#Start spark session
spark = SparkSession \
    .builder \
    .appName("ICT330 ECA Q3") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/25 20:34:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/25 20:34:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
#Read tsv file and store into dataframe
vehiclempg = spark\
.read\
.option("inferSchema", "true")\
.option("header", "true")\
.option("sep", "\t")\
.csv ("vehicle_mpg.tsv")

In [4]:
#Show the content of flights_data
vehiclempg.show(truncate=False)

+----+---------+------------+----------+------+------------+---------+------+----------------------------+
|mpg |cylinders|displacement|horsepower|weight|acceleration|modelyear|origin|carname                     |
+----+---------+------------+----------+------+------------+---------+------+----------------------------+
|18.0|8        |307.0       |130.0     |3504  |12.0        |70       |1     |chevrolet chevelle malibu   |
|15.0|8        |350.0       |165.0     |3693  |11.5        |70       |1     |buick skylark 320           |
|18.0|8        |318.0       |150.0     |3436  |11.0        |70       |1     |plymouth satellite          |
|16.0|8        |304.0       |150.0     |3433  |12.0        |70       |1     |amc rebel sst               |
|17.0|8        |302.0       |140.0     |3449  |10.5        |70       |1     |ford torino                 |
|15.0|8        |429.0       |198.0     |4341  |10.0        |70       |1     |ford galaxie 500            |
|14.0|8        |454.0       |220.0   

In [5]:
#Show the number of occurences 
vehiclempg.count()

398

In [6]:
#Show the schema
vehiclempg.printSchema()

root
 |-- mpg: double (nullable = true)
 |-- cylinders: integer (nullable = true)
 |-- displacement: double (nullable = true)
 |-- horsepower: double (nullable = true)
 |-- weight: decimal(4,0) (nullable = true)
 |-- acceleration: double (nullable = true)
 |-- modelyear: integer (nullable = true)
 |-- origin: integer (nullable = true)
 |-- carname: string (nullable = true)



In [7]:
#Show the dimensions of the Dateframe
print((vehiclempg.count(), len(vehiclempg.columns)))

(398, 9)


## 3b

In [8]:
#Display rows with missing values 
missing_data = vehiclempg.where(reduce(lambda x, y: x | y, (f.col(x).isNull() for x in vehiclempg.columns)))
missing_data.show()

+----+---------+------------+----------+------+------------+---------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|modelyear|origin|             carname|
+----+---------+------------+----------+------+------------+---------+------+--------------------+
|25.0|        4|        98.0|      null|  2046|        19.0|       71|     1|          ford pinto|
|21.0|        6|       200.0|      null|  2875|        17.0|       74|     1|       ford maverick|
|40.9|        4|        85.0|      null|  1835|        17.3|       80|     2|renault lecar deluxe|
|23.6|        4|       140.0|      null|  2905|        14.3|       80|     1|  ford mustang cobra|
|34.5|        4|       100.0|      null|  2320|        15.8|       81|     2|         renault 18i|
|23.0|        4|       151.0|      null|  3035|        20.5|       82|     1|      amc concord dl|
+----+---------+------------+----------+------+------------+---------+------+--------------------+



In [9]:
#Display the count of missing values
missing_data.count()

6

In [10]:
#Drop missing values 
vehiclempg_new = vehiclempg.na.drop()

#Show the count of rows in the original and cleaned DataFrames
print("Original DataFrame Count:", vehiclempg.count())
print("Cleaned DataFrame Count:", vehiclempg_new.count())

Original DataFrame Count: 398
Cleaned DataFrame Count: 392


In [11]:
#Show dataframe after dropping of rows
vehiclempg_new.show(truncate=False)

+----+---------+------------+----------+------+------------+---------+------+----------------------------+
|mpg |cylinders|displacement|horsepower|weight|acceleration|modelyear|origin|carname                     |
+----+---------+------------+----------+------+------------+---------+------+----------------------------+
|18.0|8        |307.0       |130.0     |3504  |12.0        |70       |1     |chevrolet chevelle malibu   |
|15.0|8        |350.0       |165.0     |3693  |11.5        |70       |1     |buick skylark 320           |
|18.0|8        |318.0       |150.0     |3436  |11.0        |70       |1     |plymouth satellite          |
|16.0|8        |304.0       |150.0     |3433  |12.0        |70       |1     |amc rebel sst               |
|17.0|8        |302.0       |140.0     |3449  |10.5        |70       |1     |ford torino                 |
|15.0|8        |429.0       |198.0     |4341  |10.0        |70       |1     |ford galaxie 500            |
|14.0|8        |454.0       |220.0   

In [12]:
#Perform statistical profiling for numerical columns
vehiclempg_new.select("mpg","cylinders","displacement","horsepower","weight","acceleration","modelyear","origin").summary().show()

23/10/25 20:35:16 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+
|summary|              mpg|         cylinders|      displacement|        horsepower|           weight|      acceleration|        modelyear|            origin|
+-------+-----------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+
|  count|              392|               392|               392|               392|              392|               392|              392|               392|
|   mean|23.44591836734694| 5.471938775510204|194.41198979591837|104.46938775510205|        2977.5842|15.541326530612228| 75.9795918367347|1.5765306122448979|
| stddev|7.805007486571802|1.7057832474527845|104.64400390890465| 38.49115993282846|849.4025600429486|  2.75886411918808|3.683736543577868|0.8055181834183057|
|    min|              9.0|                 3|

## 3c

In [14]:
#Bullet Point 1

#Create new "manufacturer" column.
#Populate the column by extracting the first word of "carname" column

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def populate_column(text):
    first_word = text.split()
    if len(first_word) > 0:
        return first_word[0]
    return None

populate_column_udf = udf(populate_column, StringType())

vehiclempg_new = vehiclempg_new.withColumn("manufacturer", populate_column_udf(col("carname")))
vehiclempg_new.show()


+----+---------+------------+----------+------+------------+---------+------+--------------------+------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|modelyear|origin|             carname|manufacturer|
+----+---------+------------+----------+------+------------+---------+------+--------------------+------------+
|18.0|        8|       307.0|     130.0|  3504|        12.0|       70|     1|chevrolet chevell...|   chevrolet|
|15.0|        8|       350.0|     165.0|  3693|        11.5|       70|     1|   buick skylark 320|       buick|
|18.0|        8|       318.0|     150.0|  3436|        11.0|       70|     1|  plymouth satellite|    plymouth|
|16.0|        8|       304.0|     150.0|  3433|        12.0|       70|     1|       amc rebel sst|         amc|
|17.0|        8|       302.0|     140.0|  3449|        10.5|       70|     1|         ford torino|        ford|
|15.0|        8|       429.0|     198.0|  4341|        10.0|       70|     1|    ford galaxie 500|      

In [15]:
vehiclempg_new.count()

392

In [16]:
#Bullet Point 2

#appending prefix "19" in data of "modelyear" column
vehiclempg_new = vehiclempg_new.withColumn("modelyear", f.concat(f.lit("19"),vehiclempg_new.modelyear))
vehiclempg_new.show()

+----+---------+------------+----------+------+------------+---------+------+--------------------+------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|modelyear|origin|             carname|manufacturer|
+----+---------+------------+----------+------+------------+---------+------+--------------------+------------+
|18.0|        8|       307.0|     130.0|  3504|        12.0|     1970|     1|chevrolet chevell...|   chevrolet|
|15.0|        8|       350.0|     165.0|  3693|        11.5|     1970|     1|   buick skylark 320|       buick|
|18.0|        8|       318.0|     150.0|  3436|        11.0|     1970|     1|  plymouth satellite|    plymouth|
|16.0|        8|       304.0|     150.0|  3433|        12.0|     1970|     1|       amc rebel sst|         amc|
|17.0|        8|       302.0|     140.0|  3449|        10.5|     1970|     1|         ford torino|        ford|
|15.0|        8|       429.0|     198.0|  4341|        10.0|     1970|     1|    ford galaxie 500|      

In [17]:
vehiclempg_new.count()

392

In [18]:
#Bullet point 3

#Create a new column called “mpg_class” with the following classification
#criteria: “low” for mpg <=20, “mid” for 20 < mpg <=30, “high” for 30 < mpg
#<=40, and “very high” for mpg > 40.

vehiclempg_new = vehiclempg_new.withColumn("mpg_class",
                when((col("mpg") <= 20) , "low")
                  .when((col("mpg") > 20) & (col("mpg") <= 30), "mid")
                  .when((col("mpg") > 30) & (col("mpg") <= 40), "high")
                  .otherwise("very high"))

vehiclempg_new.show()

+----+---------+------------+----------+------+------------+---------+------+--------------------+------------+---------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|modelyear|origin|             carname|manufacturer|mpg_class|
+----+---------+------------+----------+------+------------+---------+------+--------------------+------------+---------+
|18.0|        8|       307.0|     130.0|  3504|        12.0|     1970|     1|chevrolet chevell...|   chevrolet|      low|
|15.0|        8|       350.0|     165.0|  3693|        11.5|     1970|     1|   buick skylark 320|       buick|      low|
|18.0|        8|       318.0|     150.0|  3436|        11.0|     1970|     1|  plymouth satellite|    plymouth|      low|
|16.0|        8|       304.0|     150.0|  3433|        12.0|     1970|     1|       amc rebel sst|         amc|      low|
|17.0|        8|       302.0|     140.0|  3449|        10.5|     1970|     1|         ford torino|        ford|      low|
|15.0|        8|       4

In [19]:
vehiclempg_new.count()

392

## 3d

In [20]:
#Read csv and store in dataframe
vehiclemanufacturers = spark\
.read\
.option("inferSchema", "true")\
.option("header", "true")\
.csv ("vehicle_manufacturers.csv")

In [21]:
vehiclemanufacturers.show()

+-------------+-------+
| manufacturer|country|
+-------------+-------+
|          amc|     us|
|         audi|germany|
|          bmw|germany|
|        buick|     us|
|     cadillac|     us|
|        capri|     us|
|    chevroelt|     us|
|    chevrolet|     us|
|        chevy|     us|
|     chrysler|     us|
|       datsun|  japan|
|        dodge|     us|
|         fiat|  italy|
|         ford|     us|
|           hi|     us|
|        honda|  japan|
|        maxda|  japan|
|        mazda|  japan|
|     mercedes|germany|
|mercedes-benz|germany|
+-------------+-------+
only showing top 20 rows



In [22]:
#Join with vehiclempg_new and show content
vehicle = vehiclemanufacturers.join(vehiclempg_new,
               on='manufacturer',how="inner")
vehicle.show()

+------------+-------+----+---------+------------+----------+------+------------+---------+------+--------------------+---------+
|manufacturer|country| mpg|cylinders|displacement|horsepower|weight|acceleration|modelyear|origin|             carname|mpg_class|
+------------+-------+----+---------+------------+----------+------+------------+---------+------+--------------------+---------+
|   chevrolet|     us|18.0|        8|       307.0|     130.0|  3504|        12.0|     1970|     1|chevrolet chevell...|      low|
|       buick|     us|15.0|        8|       350.0|     165.0|  3693|        11.5|     1970|     1|   buick skylark 320|      low|
|    plymouth|     us|18.0|        8|       318.0|     150.0|  3436|        11.0|     1970|     1|  plymouth satellite|      low|
|         amc|     us|16.0|        8|       304.0|     150.0|  3433|        12.0|     1970|     1|       amc rebel sst|      low|
|        ford|     us|17.0|        8|       302.0|     140.0|  3449|        10.5|     1970

## 3e

In [24]:
#Bullet Point 1
#Find average, minimum and maximum mpg for a given country. Sort the results by average mpg descending

vehicle.groupBy("country") \
  .agg(
        f.avg(f.col('mpg')).alias('avg_mpg'),
        f.min(f.col('mpg')).alias('min_mpg'),
        f.max(f.col('mpg')).alias('max_mpg'),
    ) \
  .orderBy('avg_mpg', ascending=False) \
  .withColumn("avg_mpg", f.round(col('avg_mpg'), 2)) \
  .show()

+-------+-------+-------+-------+
|country|avg_mpg|min_mpg|max_mpg|
+-------+-------+-------+-------+
|     uk|   35.0|   35.0|   35.0|
|  japan|  30.45|   18.0|   46.6|
|germany|  29.21|   16.5|   44.3|
|  italy|  28.91|   24.0|   37.3|
| france|  25.32|   16.2|   36.0|
| sweden|  22.23|   17.0|   30.7|
|     us|  20.03|    9.0|   39.0|
+-------+-------+-------+-------+



In [25]:
#Bullet Point 2
#Find the average, minimum and maximum mpg for a given cylinder. Sort the results by average mpg descending.

vehicle.groupBy("cylinders") \
  .agg(
        f.avg(f.col('mpg')).alias('avg_mpg'),
        f.min(f.col('mpg')).alias('min_mpg'),
        f.max(f.col('mpg')).alias('max_mpg'),
    ) \
  .orderBy('avg_mpg', ascending=False) \
  .withColumn("avg_mpg", f.round(col('avg_mpg'), 2)) \
  .show()

+---------+-------+-------+-------+
|cylinders|avg_mpg|min_mpg|max_mpg|
+---------+-------+-------+-------+
|        4|  29.28|   18.0|   46.6|
|        5|  27.37|   20.3|   36.4|
|        3|  20.55|   18.0|   23.7|
|        6|  19.97|   15.0|   38.0|
|        8|  14.96|    9.0|   26.6|
+---------+-------+-------+-------+



In [26]:
#Bullet Point 3
#Find the average, minimum and maximum mpg for a given model year. Sort the results by average mpg descending.

vehicle.groupBy("modelyear") \
  .agg(
        f.avg(f.col('mpg')).alias('avg_mpg'),
        f.min(f.col('mpg')).alias('min_mpg'),
        f.max(f.col('mpg')).alias('max_mpg'),
    ) \
  .orderBy('avg_mpg', ascending=False) \
  .withColumn("avg_mpg", f.round(col('avg_mpg'), 2)) \
  .show()

+---------+-------+-------+-------+
|modelyear|avg_mpg|min_mpg|max_mpg|
+---------+-------+-------+-------+
|     1980|   33.8|   19.1|   46.6|
|     1982|   32.0|   22.0|   44.0|
|     1981|  30.19|   17.6|   39.1|
|     1979|  25.09|   15.5|   37.3|
|     1978|  24.06|   16.2|   43.1|
|     1977|  23.38|   15.0|   36.0|
|     1974|  22.77|   13.0|   32.0|
|     1976|  21.57|   13.0|   33.0|
|     1971|  21.11|   12.0|   35.0|
|     1975|  20.27|   13.0|   33.0|
|     1972|  18.71|   11.0|   28.0|
|     1970|  17.69|    9.0|   27.0|
|     1973|   17.1|   11.0|   29.0|
+---------+-------+-------+-------+



In [27]:
#Bullet Point 4
#Find the average, minimum and maximum mpg for a given manufacturer. Sort the results by average mpg descending.

vehicle.groupBy("manufacturer") \
  .agg(
        f.avg(f.col('mpg')).alias('avg_mpg'),
        f.min(f.col('mpg')).alias('min_mpg'),
        f.max(f.col('mpg')).alias('max_mpg'),
    ) \
  .orderBy('avg_mpg', ascending=False) \
  .withColumn("avg_mpg", f.round(col('avg_mpg'), 2)) \
  .show()

+------------+-------+-------+-------+
|manufacturer|avg_mpg|min_mpg|max_mpg|
+------------+-------+-------+-------+
|          vw|  39.02|   29.0|   44.3|
|      nissan|   36.0|   36.0|   36.0|
|     triumph|   35.0|   35.0|   35.0|
|       honda|  33.76|   24.0|   44.6|
|      datsun|  31.11|   22.0|   40.8|
|       mazda|  30.86|   19.0|   46.6|
|      subaru|  30.53|   26.0|   33.8|
|   vokswagen|   29.8|   29.8|   29.8|
|     renault|  29.67|   26.0|   36.0|
|  volkswagen|  29.11|   22.0|   43.1|
|        fiat|  28.91|   24.0|   37.3|
|      toyota|  28.37|   19.0|   39.1|
|        audi|  26.71|   20.0|   36.4|
|       maxda|  26.05|   18.0|   34.1|
|        opel|  25.75|   24.0|   28.0|
|    mercedes|   25.4|   25.4|   25.4|
|       capri|   25.0|   25.0|   25.0|
|        saab|   23.9|   21.6|   25.0|
|         bmw|  23.75|   21.5|   26.0|
|     peugeot|  23.69|   16.2|   30.0|
+------------+-------+-------+-------+
only showing top 20 rows



In [28]:
#Bullet Point 5
#Find the average mpg for a given car name and manufacturer. Sort the results by average mpg descending.

vehicle.groupBy("manufacturer","carname") \
  .agg(f.avg(f.col('mpg')).alias('avg_mpg')) \
  .orderBy('avg_mpg', ascending=False) \
  .withColumn("avg_mpg", f.round(col('avg_mpg'), 2)) \
  .show()

+------------+--------------------+-------+
|manufacturer|             carname|avg_mpg|
+------------+--------------------+-------+
|       mazda|           mazda glc|   46.6|
|       honda| honda civic 1500 gl|   44.6|
|          vw|vw rabbit c (diesel)|   44.3|
|          vw|           vw pickup|   44.0|
|          vw|  vw dasher (diesel)|   43.4|
|  volkswagen|volkswagen rabbit...|   43.1|
|      datsun|      datsun b210 gx|   39.4|
|      toyota|      toyota starlet|   39.1|
|    plymouth|      plymouth champ|   39.0|
|      toyota|toyota corolla te...|   38.1|
|    plymouth|plymouth horizon ...|   38.0|
|      datsun|       datsun 310 gx|   38.0|
|  oldsmobile|oldsmobile cutlas...|   38.0|
|      toyota|       toyota tercel|   37.7|
|        fiat|  fiat strada custom|   37.3|
|      datsun|          datsun 310|   37.2|
|      datsun|      datsun 210 mpg|   37.0|
|       mazda|  mazda glc custom l|   37.0|
|      datsun|datsun 510 hatchback|   37.0|
|        audi| audi 5000s (diese

## 3f

In [29]:
vehicle.createOrReplaceTempView("vehicle")

In [31]:
#Qn 3e Bullet Point 1
#Find average, minimum and maximum mpg for a given country. Sort the results by average mpg descending

spark.sql("""
SELECT country, ROUND(AVG(mpg), 2) AS average_mpg, MIN(mpg) AS min_mpg, MAX(mpg) AS max_mpg
FROM vehicle
GROUP BY country
ORDER BY average_mpg DESC
""").show()

+-------+-----------+-------+-------+
|country|average_mpg|min_mpg|max_mpg|
+-------+-----------+-------+-------+
|     uk|       35.0|   35.0|   35.0|
|  japan|      30.45|   18.0|   46.6|
|germany|      29.21|   16.5|   44.3|
|  italy|      28.91|   24.0|   37.3|
| france|      25.32|   16.2|   36.0|
| sweden|      22.23|   17.0|   30.7|
|     us|      20.03|    9.0|   39.0|
+-------+-----------+-------+-------+



In [32]:
#Qn 3e Bullet Point 2
#Find the average, minimum and maximum mpg for a given cylinder. Sort the results by average mpg descending.

spark.sql("""
SELECT cylinders, ROUND(AVG(mpg), 2) AS average_mpg, MIN(mpg) AS min_mpg, MAX(mpg) AS max_mpg
FROM vehicle
GROUP BY cylinders
ORDER BY average_mpg DESC
""").show()

+---------+-----------+-------+-------+
|cylinders|average_mpg|min_mpg|max_mpg|
+---------+-----------+-------+-------+
|        4|      29.28|   18.0|   46.6|
|        5|      27.37|   20.3|   36.4|
|        3|      20.55|   18.0|   23.7|
|        6|      19.97|   15.0|   38.0|
|        8|      14.96|    9.0|   26.6|
+---------+-----------+-------+-------+



In [33]:
#Qn 3e Bullet Point 3
#Find the average, minimum and maximum mpg for a given model year. Sort the results by average mpg descending.

spark.sql("""
SELECT modelyear, ROUND(AVG(mpg), 2) AS average_mpg, MIN(mpg) AS min_mpg, MAX(mpg) AS max_mpg
FROM vehicle
GROUP BY modelyear
ORDER BY average_mpg DESC
""").show()

+---------+-----------+-------+-------+
|modelyear|average_mpg|min_mpg|max_mpg|
+---------+-----------+-------+-------+
|     1980|       33.8|   19.1|   46.6|
|     1982|       32.0|   22.0|   44.0|
|     1981|      30.19|   17.6|   39.1|
|     1979|      25.09|   15.5|   37.3|
|     1978|      24.06|   16.2|   43.1|
|     1977|      23.38|   15.0|   36.0|
|     1974|      22.77|   13.0|   32.0|
|     1976|      21.57|   13.0|   33.0|
|     1971|      21.11|   12.0|   35.0|
|     1975|      20.27|   13.0|   33.0|
|     1972|      18.71|   11.0|   28.0|
|     1970|      17.69|    9.0|   27.0|
|     1973|       17.1|   11.0|   29.0|
+---------+-----------+-------+-------+



In [34]:
#Qn 3e Bullet Point 4
#Find the average, minimum and maximum mpg for a given manufacturer. Sort the results by average mpg descending.

spark.sql("""
SELECT manufacturer, ROUND(AVG(mpg), 2) AS average_mpg, MIN(mpg) AS min_mpg, MAX(mpg) AS max_mpg
FROM vehicle
GROUP BY manufacturer
ORDER BY average_mpg DESC
""").show()

+------------+-----------+-------+-------+
|manufacturer|average_mpg|min_mpg|max_mpg|
+------------+-----------+-------+-------+
|          vw|      39.02|   29.0|   44.3|
|      nissan|       36.0|   36.0|   36.0|
|     triumph|       35.0|   35.0|   35.0|
|       honda|      33.76|   24.0|   44.6|
|      datsun|      31.11|   22.0|   40.8|
|       mazda|      30.86|   19.0|   46.6|
|      subaru|      30.53|   26.0|   33.8|
|   vokswagen|       29.8|   29.8|   29.8|
|     renault|      29.67|   26.0|   36.0|
|  volkswagen|      29.11|   22.0|   43.1|
|        fiat|      28.91|   24.0|   37.3|
|      toyota|      28.37|   19.0|   39.1|
|        audi|      26.71|   20.0|   36.4|
|       maxda|      26.05|   18.0|   34.1|
|        opel|      25.75|   24.0|   28.0|
|    mercedes|       25.4|   25.4|   25.4|
|       capri|       25.0|   25.0|   25.0|
|        saab|       23.9|   21.6|   25.0|
|         bmw|      23.75|   21.5|   26.0|
|     peugeot|      23.69|   16.2|   30.0|
+----------

In [35]:
#Qn 3e Bullet Point 5
#Find the average mpg for a given car name and manufacturer. Sort the results by average mpg descending.

spark.sql("""
SELECT manufacturer, carname, ROUND(AVG(mpg), 2) AS average_mpg
FROM vehicle
GROUP BY manufacturer,carname
ORDER BY average_mpg DESC
""").show()

+------------+--------------------+-----------+
|manufacturer|             carname|average_mpg|
+------------+--------------------+-----------+
|       mazda|           mazda glc|       46.6|
|       honda| honda civic 1500 gl|       44.6|
|          vw|vw rabbit c (diesel)|       44.3|
|          vw|           vw pickup|       44.0|
|          vw|  vw dasher (diesel)|       43.4|
|  volkswagen|volkswagen rabbit...|       43.1|
|      datsun|      datsun b210 gx|       39.4|
|      toyota|      toyota starlet|       39.1|
|    plymouth|      plymouth champ|       39.0|
|      toyota|toyota corolla te...|       38.1|
|    plymouth|plymouth horizon ...|       38.0|
|      datsun|       datsun 310 gx|       38.0|
|  oldsmobile|oldsmobile cutlas...|       38.0|
|      toyota|       toyota tercel|       37.7|
|        fiat|  fiat strada custom|       37.3|
|      datsun|          datsun 310|       37.2|
|      datsun|      datsun 210 mpg|       37.0|
|       mazda|  mazda glc custom l|     

In [36]:
#Stop Spark Session
spark.stop()